In [1]:
import DSGRN
from DSGRN import *
import cProfile
import sys
sys.setrecursionlimit(10**8)
sys.path.insert(0,'/home/elizabeth/Desktop/GIT/dsgrn_acdc/src')

from PhenotypeGraphviz import *
from PhenotypeGraphFun import *
from ReducePhenotypeGraph import *

In [2]:
database = Database("/home/elizabeth/Desktop/ACDC/ACDC_FullconnE.db") 
network = Network("/home/elizabeth/Desktop/ACDC/ACDC_FullconnE") 

In [3]:
parameter_graph = ParameterGraph(network)
print(parameter_graph.size())

38416


In [4]:
AP35 = {"Hb":[0,2], "Gt":2, "Kr":0, "Kni":0}
AP37 = {"Hb":2, "Gt":[0,1], "Kr":0, "Kni":0}
AP40 = {"Hb":2, "Gt":1, "Kr":[0,1], "Kni":0} #edit
AP45 = {"Hb":[0,1], "Gt":1, "Kr":2, "Kni":0} #edit
AP47 = {"Hb":[0,1], "Gt":0, "Kr":2, "Kni":0} 
AP51 = {"Hb":1, "Gt":0, "Kr":2, "Kni":[0,1]} #edit
AP57 = {"Hb":1, "Gt":0, "Kr":[0,1], "Kni":2} #edit
AP61 = {"Hb":0, "Gt":0, "Kr":[0,1], "Kni":2}
AP63 = {"Hb":0, "Gt":[0,1], "Kr":1, "Kni":2} #edit
AP67 = {"Hb":0, "Gt":2, "Kr":1, "Kni":[0,1]} #edit

In [5]:
D = [[AP37], [AP40], [AP45], [AP47],[AP51], [AP57], [AP61], [AP63], [AP67]]
paramslist = get_paramslist(database, D, '=')

In [6]:
cProfile.runctx('get_paramslist(database, list_of_bounds, goe)', globals(), {'database':database,'list_of_bounds':D, 'goe':'='})

         330009 function calls (275181 primitive calls) in 0.252 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.252    0.252 <string>:1(<module>)
       45    0.000    0.000    0.000    0.000 FixedPointTables.py:12(FPString)
       45    0.000    0.000    0.000    0.000 FixedPointTables.py:13(<listcomp>)
        9    0.000    0.000    0.000    0.000 FixedPointTables.py:18(buildQueryExpression)
       36    0.000    0.000    0.000    0.000 FixedPointTables.py:32(<listcomp>)
        9    0.000    0.000    0.011    0.001 FixedPointTables.py:36(MatchQuery)
       63    0.000    0.000    0.001    0.000 Logging.py:2(LogToSTDOUT)
        9    0.000    0.000    0.000    0.000 NFixedPointQuery.py:14(__enter__)
        9    0.000    0.000    0.000    0.000 NFixedPointQuery.py:18(__exit__)
        9    0.000    0.000    0.015    0.002 NFixedPointQuery.py:24(__init__)
        9    0.000    0.000    0.0

In [7]:
edges = get_phenotype_graph(database, paramslist)
edges

{(0, 14): [(1, 21),
  (0, 19222),
  (1, 19222),
  (0, 21),
  (0, 2415),
  (1, 16),
  (1, 14),
  (0, 9618),
  (1, 2415),
  (1, 9618),
  (0, 16),
  (0, 63),
  (1, 63)],
 (0, 21): [(1, 21),
  (0, 2422),
  (1, 2422),
  (1, 9625),
  (1, 23),
  (0, 9625),
  (0, 14),
  (1, 14),
  (0, 70),
  (1, 70),
  (1, 28),
  (1, 19229),
  (0, 19229),
  (0, 23),
  (0, 28)],
 (0, 28): [(1, 21),
  (0, 9632),
  (0, 21),
  (1, 77),
  (0, 77),
  (1, 19236),
  (0, 30),
  (1, 28),
  (0, 19236),
  (1, 2429),
  (1, 9632),
  (0, 2429),
  (1, 30)],
 (0, 63): [(0, 65),
  (0, 2464),
  (0, 14),
  (1, 14),
  (0, 19271),
  (0, 70),
  (1, 70),
  (1, 19271),
  (1, 2464),
  (1, 65),
  (1, 63)],
 (0, 67): [(1, 67),
  (0, 410),
  (1, 410),
  (0, 18),
  (1, 18),
  (0, 19275),
  (0, 74),
  (1, 74),
  (1, 19275)],
 (0, 70): [(1, 21),
  (0, 21),
  (1, 77),
  (0, 19278),
  (1, 19278),
  (0, 77),
  (0, 2471),
  (1, 72),
  (1, 70),
  (0, 63),
  (1, 2471),
  (0, 72),
  (1, 63)],
 (0, 74): [(0, 417),
  (1, 67),
  (1, 25),
  (0, 25),
  

In [8]:
cProfile.runctx('get_phenotype_graph(database, paramslist)', globals(), {'database':database,'paramslist':paramslist})

         31434 function calls in 0.791 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.795    0.795 <string>:1(<module>)
        1    0.480    0.480    0.793    0.793 PhenotypeGraphFun.py:53(get_phenotype_graph)
        9    0.000    0.000    0.000    0.000 PhenotypeGraphFun.py:63(<listcomp>)
        1    0.001    0.001    0.001    0.001 PhenotypeGraphFun.py:68(<listcomp>)
     4056    0.067    0.000    0.067    0.000 PhenotypeGraphFun.py:76(<listcomp>)
     1179    0.058    0.000    0.058    0.000 PhenotypeGraphFun.py:78(<listcomp>)
     5235    0.172    0.000    0.172    0.000 PhenotypeGraphFun.py:80(<listcomp>)
     5235    0.002    0.000    0.002    0.000 PhenotypeGraphFun.py:85(<listcomp>)
        1    0.000    0.000    0.795    0.795 {built-in method builtins.exec}
     5235    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000

In [9]:
[a,b,c] = reduce(edges, paramslist)

In [10]:
cProfile.runctx('reduce(edges, paramslist)', globals(), {'edges':edges,'paramslist':paramslist})

         2600835 function calls (2185544 primitive calls) in 3.667 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    3.667    3.667 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 PhenotypeGraphFun.py:9(flatten)
        9    0.001    0.000    0.001    0.000 ReducePhenotypeGraph.py:114(<listcomp>)
        9    0.210    0.023    0.210    0.023 ReducePhenotypeGraph.py:115(<listcomp>)
    38003    0.008    0.000    0.009    0.000 ReducePhenotypeGraph.py:13(add_edge)
        1    0.006    0.006    0.007    0.007 ReducePhenotypeGraph.py:17(add_edges)
  5235/18    0.004    0.000    0.004    0.000 ReducePhenotypeGraph.py:24(dfs)
   5235/8    0.005    0.000    0.005    0.001 ReducePhenotypeGraph.py:32(fill_order)
        1    0.004    0.004    0.014    0.014 ReducePhenotypeGraph.py:42(transpose)
        1    0.002    0.002    0.335    0.335 ReducePhenotypeGraph.py:51(scc)
        2    